In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import requests

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

In [3]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

In [4]:
HOME_PAGE = "https://thuvienphapluat.vn/hoi-dap-phap-luat/thue-phi-le-phi"
PAGE_NAME = "thuvienphapluat"

# Crawl data

## Get urls of the documents

In [6]:
MAX_DOCUMENT = 2000

driver = webdriver.Chrome(options=chrome_options)
driver.get(HOME_PAGE)

In [7]:
documents = driver.find_elements(by=By.CLASS_NAME, value="news-card")

urls = []
while len(urls) < MAX_DOCUMENT:
    for doc in documents:
        if len(urls) < MAX_DOCUMENT:
            url = doc.find_element(by=By.TAG_NAME, value="a").get_attribute("href")
            urls.append(url)
        else:
            break

    next_page = driver.find_element(by=By.CLASS_NAME, value="page-link").get_attribute("href")
    documents = driver.find_elements(by=By.CLASS_NAME, value="news-card")


In [8]:
len(urls), urls[:10]

(2000,
 ['https://thuvienphapluat.vn/hoi-dap-phap-luat/83A3AEA-hd-tai-ve-mau-so-22-thong-bao-hoan-thanh-thu-tuc-hai-quan-nhap-khau-tai-cho-moi-nhat.html',
  'https://thuvienphapluat.vn/hoi-dap-phap-luat/83A3A41-hd-ai-co-tham-quyen-quyet-dinh-cuong-che-bang-bien-phap-ngung-su-dung-hoa-don.html',
  'https://thuvienphapluat.vn/hoi-dap-phap-luat/83A3A6E-hd-ty-le-thue-tndn-cua-to-chuc-nuoc-ngoai-doi-voi-nganh-kinh-doanh-cung-cap-hang-hoa-theo-dieu-kien-gi.html',
  'https://thuvienphapluat.vn/hoi-dap-phap-luat/83A3A98-hd-mau-so-chi-tiet-thue-gtgt-duoc-mien-giam-dung-cho-doanh-nghiep-nho-va-vua-moi-nhat-2024.html',
  'https://thuvienphapluat.vn/hoi-dap-phap-luat/83A3A63-hd-ho-tro-nguoi-lao-dong-1-den-3-trieu-dongtruong-hop-thiet-hai-do-bao-so-3-bao-yagi.html',
  'https://thuvienphapluat.vn/hoi-dap-phap-luat/83A3A6C-hd-cach-viet-so-tien-bang-chu-tren-hoa-don-theo-nghi-dinh-123.html',
  'https://thuvienphapluat.vn/hoi-dap-phap-luat/83A3931-hd-mau-15bbbkd-bien-ban-xac-minh-tinh-trang-hoat-dong-c

## Get the content of each urls

In [9]:
def get_content_url(driver, url):
    driver.get(url)
    title = driver.find_element(by=By.CLASS_NAME, value="title").text
    description = driver.find_element(by=By.CLASS_NAME, value="sapo").text
    contents = driver.find_element(by=By.CLASS_NAME, value="news-content").find_elements(by=By.XPATH, value="./*")

    count = 0
    sub_contents = []
    small_contents = {}
    sub_content = []
    except_tagname = ['div', 'table', 'strong']
    for i, content in enumerate(contents):
        if content.tag_name == "h2" and content.tag_name not in except_tagname:
            if len(sub_content) > 0:
                small_contents.update({
                    "sub_content": sub_content
                })
                sub_contents.append(small_contents)
                small_contents = {}
                sub_content = []
            ct = content.text
            small_contents.update({
                    "sub_title": ct,
                    "sub_content": sub_content
                })
        elif content.tag_name not in except_tagname:
            ct = content.text
            sub_content.append(ct)
        else:
            continue

        if i == len(contents) - 1:
            small_contents.update({
                "sub_content": sub_content
            })
            sub_contents.append(small_contents)
    contents = {
        "url": url,
        "title": title,
        "description": description,
        "content": sub_contents
    }
    return contents

In [10]:
url = urls[0]
content = get_content_url(driver=driver, url=url)

In [11]:
content

{'url': 'https://thuvienphapluat.vn/hoi-dap-phap-luat/83A3AEA-hd-tai-ve-mau-so-22-thong-bao-hoan-thanh-thu-tuc-hai-quan-nhap-khau-tai-cho-moi-nhat.html',
 'title': 'Tải về mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ mới nhất?',
 'description': 'Tải về mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ mới nhất?',
 'content': [{'sub_title': 'Mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ mới nhất?',
   'sub_content': ['Tại Nghị định 18/2021/NĐ-CP có quy định mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ.',
    'Sau đây là mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ:',
    '',
    'Xem chi tiết mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ Tải về.']},
  {'sub_title': 'Thời hạn làm thủ tục hải quan đối với hàng hóa xuất khẩu, nhập khẩu tại chỗ là khi nào?',
   'sub_content': ['Tại khoản 4 Điều 86 Thông tư 38/2015/TT-BTC được sửa đổi bởi khoản 58 Điều 1 Thông tư 39/2018/TT-

In [ ]:
from tqdm import tqdm
contents = []
count = 0
for url in tqdm(urls):
  try:
    if count < MAX_DOCUMENT:
      content = get_content_url(driver=driver, url=url)
      contents.append(content)
      count += 1
    else:
      break
  except (StaleElementReferenceException) as e:
      print(f"Bug at url: {url}, with StaleElementReferenceException")
      driver.refresh()
      continue

  except (NoSuchElementException) as e:
      print(f"Bug at url: {url}, with NoSuchElementException")
      driver.refresh()
      continue

with open('data_crawl.json', 'w', encoding='utf-8') as f:
    json.dump(contents, f, ensure_ascii=False, indent=4)

driver.close()

# Preprocessing data

## Devide the content per documents into the sub-contents

In [6]:
with open('data_crawl.json', 'r', encoding='utf-8') as f:
    contents = json.load(f)

In [7]:
all_contents = []
for content in contents:
    all_contents.extend(content['content'])

In [9]:
print(f"Length of contents before devide: {len(contents)}")
print(f"Length of contents after devide: {len(all_contents)}")
all_contents[10]

Length of contents before devide: 2000
Length of contents after devide: 6000


{'sub_title': 'Doanh nghiệp nhỏ và vừa được thay đổi đơn vị tiền tệ trong kế toán khi nào?',
 'sub_content': ['Căn cứ theo Điều 8 Thông tư 133/2016/TT-BTC quy định như sau:',
  'Điều 8. Thay đổi đơn vị tiền tệ trong kế toán',
  'Khi có sự thay đổi lớn về hoạt động quản lý và kinh doanh dẫn đến đơn vị tiền tệ kế toán được sử dụng trong các giao dịch kinh tế không còn thỏa mãn các tiêu chuẩn tại khoản 2, 3 Điều 6 Thông tư này thì doanh nghiệp được thay đổi đơn vị tiền tệ trong kế toán. Việc thay đổi từ một đơn vị tiền tệ ghi sổ kế toán này sang một đơn vị tiền tệ ghi sổ kế toán khác chỉ được thực hiện tại thời điểm bắt đầu niên độ kế toán mới.',
  'Doanh nghiệp phải thông báo cho cơ quan thuế quản lý trực tiếp về việc thay đổi đơn vị tiền tệ trong kế toán chậm nhất là sau 10 ngày làm việc kể từ ngày thay đổi đơn vị tiền tệ.',
  'Như vậy, doanh nghiệp nhỏ và vừa được thay đổi đơn vị tiền tệ trong kế toán khi có sự thay đổi lớn về hoạt động quản lý và kinh doanh dẫn đến đơn vị tiền tệ kế t

## Process data


In [10]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.5 MB/s eta 0:00:00


In [11]:
import re
import string
from underthesea import word_tokenize

stop_words = set([
    "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện",
    "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều",
    "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", "này",
    "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", "qua", "ra", "rằng",
    "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", "trước", "từ", "từng",
    "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
])

def processing_text(text):
  text = text.lower()
  text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
  text = re.sub(r"\s+", " ", text).strip()
  text_tokenize = word_tokenize(text)
  tokens = [word for word in text_tokenize if word not in stop_words]
  return ' '.join(tokens) if len(tokens) > 0 else ''

In [12]:
text = all_contents[0]['sub_content'][1]
print(f"Before processing: {text}")
print(f"After processing: {processing_text(text)}")

Before processing: Sau đây là mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ:
After processing: mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ


In [13]:
print(all_contents[0])

{'sub_title': 'Mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ mới nhất?', 'sub_content': ['Tại Nghị định 18/2021/NĐ-CP có quy định mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ.', 'Sau đây là mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ:', '', 'Xem chi tiết mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ Tải về.']}


In [14]:
list(all_contents[0].values())[1]

['Tại Nghị định 18/2021/NĐ-CP có quy định mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ.',
 'Sau đây là mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ:',
 '',
 'Xem chi tiết mẫu số 22 thông báo hoàn thành thủ tục hải quan nhập khẩu tại chỗ Tải về.']

In [17]:
from tqdm import tqdm

processed_docs = []
for content in tqdm(all_contents):
  all_ct = list(content.values())
  question = all_ct[0]
  answers = all_ct[1]
  processed_question = processing_text(question)
  processed_answers = [processing_text(answer) for answer in answers if len(processing_text(answer)) > 0]
  processed_docs.append({
      "question": processed_question,
      "answers": processed_answers
  })

100%|██████████| 6000/6000 [05:47<00:00, 17.28it/s]


In [18]:
processed_docs[10]

{'question': 'doanh nghiệp nhỏ thay đổi đơn vị tiền tệ trong kế toán nào',
 'answers': ['căn cứ điều 8 thông tư 1332016 ttbtc quy định',
  'điều 8 thay đổi đơn vị tiền tệ trong kế toán',
  'thay đổi lớn về hoạt động quản lý kinh doanh dẫn đến đơn vị tiền tệ kế toán sử dụng trong giao dịch kinh tế còn thỏa mãn tiêu chuẩn khoản 2 3 điều 6 thông tư doanh nghiệp thay đổi đơn vị tiền tệ trong kế toán thay đổi một đơn vị tiền tệ ghi sổ kế toán sang một đơn vị tiền tệ ghi sổ kế toán khác thực hiện thời điểm bắt đầu niên độ kế toán mới',
  'doanh nghiệp thông báo cơ quan thuế quản lý trực tiếp về thay đổi đơn vị tiền tệ trong kế toán chậm nhất 10 ngày làm việc kể ngày thay đổi đơn vị tiền tệ',
  'như vậy doanh nghiệp nhỏ thay đổi đơn vị tiền tệ trong kế toán thay đổi lớn về hoạt động quản lý kinh doanh dẫn đến đơn vị tiền tệ kế toán sử dụng trong giao dịch kinh tế còn thỏa mãn tiêu chuẩn',
  'sử dụng chủ yếu trong giao dịch bán hàng cung cấp dịch vụ đơn vị ảnh hưởng lớn đến giá bán hàng hóa cu

In [19]:
with open('data_processed.json', 'w', encoding='utf-8') as f:
    json.dump(processed_docs, f, ensure_ascii=False, indent=4)

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
with open('/content/drive/MyDrive/AI_For_Engineer/NLP_Project/data_crawl.json', 'w', encoding='utf-8') as f:
    json.dump(contents, f, ensure_ascii=False, indent=4)


with open('/content/drive/MyDrive/AI_For_Engineer/NLP_Project/data_processed.json', 'w', encoding='utf-8') as f:
    json.dump(processed_docs, f, ensure_ascii=False, indent=4)